In [1]:
from imutils import paths
import numpy as np
import imutils
import pickle
import cv2
import os

In [2]:
# Prepare model
modelFolder = "./face_detection_model"
protoFile = os.path.sep.join([modelFolder, "deploy.prototxt"])
modelFile = os.path.sep.join([modelFolder, "res10_300x300_ssd_iter_140000.caffemodel"])
# detector = cv2.dnn.readNetFromCaffe(protoFile, modelFile)

# Prepare image dataset
dataSetFolder = "./dataset"
# imageFiles = list(paths.list_images(dataSetFolder))

# Prepare embedded
embeddedFile = "./openface_nn4.small2.v1.t7"
# embedder = cv2.dnn.readNetFromTorch(embeddedFile)

In [3]:
# Set Output
outputFile = "output/embeddings.pickle"

knownLabels = []
knownEmbeddings = []
# Prepare parameters
minimumConfidence = 0.5
faceMinWidth, faceMinHeight = 20, 20
faceOutputWidth, faceOutputHeight = 96, 96
faceScale = 1.0 / 255
faceMean = (0, 0, 0)
# Resize image to 300x300 pixels to blob object and normalizing
resizedWidth, resizedHeight = 300, 300
scale = 1.0
mean = (104.0, 177.0, 123.0)
#########################################
# Load face model 
detector = cv2.dnn.readNetFromCaffe(protoFile, modelFile)

# Load embedding model
embedder = cv2.dnn.readNetFromTorch(embeddedFile)

# List all images in dataset
imageFiles = list(paths.list_images(dataSetFolder))
totalImage = len(imageFiles)
print("totalImage={0}".format(totalImage))

# Process image dataset
for (i, imageFile) in enumerate(imageFiles):
    label = imageFile.split(os.path.sep)[-2]
    print("> processing image {0:02d}/{1:02d} => {2}".format(i + 1, totalImage, label))
    # Load image and resize while maintaining the aspect ratio
    image = cv2.imread(imageFile)
    image = imutils.resize(image, width=600)
    (orgHeight, orgWidth) = image.shape[:2]
    resizedImage = cv2.resize(image, (resizedWidth, resizedHeight))
    # Create a blob image
    inputBlob = cv2.dnn.blobFromImage(resizedImage, scale, (resizedWidth, resizedHeight), mean, swapRB=False, crop=False)

    # Apply a face detector
    detector.setInput(inputBlob)
    detections = detector.forward()

    # If there is one face found at least
    if len(detections) > 0:
        # we're making the assumption that each image has only ONE face
        k = np.argmax(detections[0, 0, :, 2])
        confidence = detections[0, 0, k, 2]
        if confidence > minimumConfidence:
            # Compute face bounding box
            box = detections[0, 0, k, 3:7] * np.array([orgWidth, orgHeight, orgWidth, orgHeight])
            (startX, startY, endX, endY) = box.astype("int")

            # Extract the face ROI
            face = image[startY:endY, startX:endX]
            (faceHeight, faceWidth) = face.shape[:2]

            # Face size is larger than the minimum
            if faceWidth < faceMinWidth or faceHeight < faceMinHeight:
                continue

            print("  - face found.  Confidence={0:.2f}".format(confidence))
            
            # Create a blob from the face ROI
            faceBlob = cv2.dnn.blobFromImage(face, faceScale, (faceOutputWidth, faceOutputHeight), faceMean, swapRB=True, crop=False)
            # Pass face blob to embedding model to obtain the 128-d quantification of the face
            embedder.setInput(faceBlob)
            vec = embedder.forward()

            # Add the detection
            knownLabels.append(label)
            knownEmbeddings.append(vec.flatten())
# Dump all embeddings to disk            
print("[INFO] serializing {0} encodings".format(len(knownEmbeddings)))
data = { "embeddings": knownEmbeddings, "labels": knownLabels }
# Write to file
print(">>> {0}".format(outputFile))
fp = open(outputFile, "wb")
fp.write(pickle.dumps(data))
fp.close()


totalImage=15
> processing image 01/15 => kitty
  - face found.  Confidence=1.00
> processing image 02/15 => kitty
  - face found.  Confidence=0.99
> processing image 03/15 => kitty
  - face found.  Confidence=1.00
> processing image 04/15 => kitty
  - face found.  Confidence=1.00
> processing image 05/15 => kitty
  - face found.  Confidence=1.00
> processing image 06/15 => kitty
  - face found.  Confidence=1.00
> processing image 07/15 => unknown
  - face found.  Confidence=0.98
> processing image 08/15 => unknown
  - face found.  Confidence=1.00
> processing image 09/15 => unknown
  - face found.  Confidence=1.00
> processing image 10/15 => unknown
  - face found.  Confidence=1.00
> processing image 11/15 => unknown
  - face found.  Confidence=1.00
> processing image 12/15 => unknown
  - face found.  Confidence=1.00
> processing image 13/15 => unknown
  - face found.  Confidence=1.00
> processing image 14/15 => unknown
  - face found.  Confidence=1.00
> processing image 15/15 => unkn